<a href="https://colab.research.google.com/github/WilliamShengYangHuang/AALU_Workshop_3/blob/main/mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
target_image_path  = '/content/drive/My Drive/untitled.png' #@param{type:'string'}
image_library_folder = '/content/drive/My Drive/Teaching/UK/AA/Landscape_pinterest/2/' #@param{type:'string'}

## Method 1: Basic Mosaic

In [ ]:
def resize_image(image, size):
    """Resize an image to the given size."""
    return image.resize(size)

def calculate_average_color(image):
    """Calculate the average color of the given image."""
    np_image = np.array(image)
    w, h, d = np_image.shape
    return np_image.reshape(w*h, d).mean(axis=0)

def apply_sharpen_filter(image):
    """Apply a sharpening filter to the image."""
    return image.filter(ImageFilter.SHARPEN)

def generate_unique_filename(base_path):
    """Generate a unique filename."""
    count = 1
    while True:
        path = f"{base_path}_{count}.png"
        if not os.path.exists(path):
            return path
        count += 1

def create_mosaic(target_image_path, image_library_folder, num_tiles_x, num_tiles_y, output_width=None, output_height=None, apply_sharpening=False):
    print("Loading target image...")
    target = Image.open(target_image_path).convert('RGB')

    # Adjusting the target image size to match the output resolution, if specified
    if output_width and output_height:
        target = target.resize((output_width, output_height))

    target_width, target_height = target.size

    # Setting tile width and height to 32 pixels
    tile_width = tile_height = 32

    print("Loading images from the library...")
    image_files = os.listdir(image_library_folder)
    images = [Image.open(os.path.join(image_library_folder, img)).convert('RGB')
              for img in image_files if img.endswith(('png', 'jpg', 'jpeg'))]
    print(f"Loaded {len(images)} images.")

    print("Processing target image...")
    target_width = num_tiles_x * tile_width
    target_height = num_tiles_y * tile_height
    target = target.resize((target_width, target_height))

    print("Resizing images in the library...")
    tile_size = (tile_width, tile_height)
    if apply_sharpening:
        images_resized = [apply_sharpen_filter(resize_image(img, tile_size)) for img in images]
    else:
        images_resized = [resize_image(img, tile_size) for img in images]
    average_colors = [calculate_average_color(img) for img in images_resized]

    print("Creating mosaic...")
    mosaic = Image.new('RGB', target.size)
    used_images = []

    for i in range(0, target_width, tile_width):
        for j in range(0, target_height, tile_height):
            target_tile = target.crop((i, j, i + tile_width, j + tile_height))
            target_color = calculate_average_color(target_tile)
            distances = [np.linalg.norm(target_color - color) for color in average_colors]
            closest_img_index = np.argmin(distances)
            closest_img = images_resized[closest_img_index]
            mosaic.paste(closest_img, (i, j))
            filename = image_files[closest_img_index]
            used_images.append(filename)
        print(f"Row {j // tile_height + 1} of {target_height // tile_height} complete.")

    if output_width and output_height:
        mosaic = mosaic.resize((output_width, output_height))

    base_output_path = '/content/drive/My Drive/mosaic'  # Change to your desired path
    output_path = generate_unique_filename(base_output_path)
    mosaic.save(output_path)
    print(f"Mosaic created successfully. Saved to {output_path}")

    # Visualization
    plt.imshow(mosaic)
    plt.axis('off')
    plt.show()

    return used_images


In [ ]:
#used_tile_filenames = create_mosaic(target_image_path, image_library_folder, tile_height=10, tile_width=10, scale_factor=4, apply_sharpening=sharpening)

used_tile_filenames = create_mosaic(target_image_path, image_library_folder, num_tiles_x=75, num_tiles_y=50, output_width=None, output_height=None, apply_sharpening=False)
print("Used tile filenames:", used_tile_filenames)

## Method 2: To avoid repetitions and maximise the diversity of images in the mosaic

In [ ]:

def resize_image(image, size):
    """Resize an image to the given size."""
    return image.resize(size)

def calculate_average_color(image):
    """Calculate the average color of the given image."""
    np_image = np.array(image)
    w, h, d = np_image.shape
    return np_image.reshape(w*h, d).mean(axis=0)

def apply_sharpen_filter(image):
    """Apply a sharpening filter to the image."""
    return image.filter(ImageFilter.SHARPEN)

def generate_unique_filename(base_path):
    """Generate a unique filename."""
    count = 1
    while True:
        path = f"{base_path}_{count}.png"
        if not os.path.exists(path):
            return path
        count += 1

def create_mosaic(target_image_path, image_library_folder, num_tiles_x, num_tiles_y, output_width=None, output_height=None, apply_sharpening=False, mosaic_size_inches=(20, 20), dpi=150):
    print("Loading target image...")
    target = Image.open(target_image_path).convert('RGB')

    if output_width and output_height:
        target = target.resize((output_width, output_height))

    target_width, target_height = target.size
    tile_width = tile_height = 32

    print("Loading images from the library...")
    image_files = os.listdir(image_library_folder)
    images = [Image.open(os.path.join(image_library_folder, img)).convert('RGB') for img in image_files if img.endswith(('png', 'jpg', 'jpeg'))]
    print(f"Loaded {len(images)} images.")

    image_usage = {img: 0 for img in image_files}

    print("Processing target image...")
    target_width = num_tiles_x * tile_width
    target_height = num_tiles_y * tile_height
    target = target.resize((target_width, target_height))

    print("Resizing images in the library...")
    tile_size = (tile_width, tile_height)
    if apply_sharpening:
        images_resized = [apply_sharpen_filter(resize_image(img, tile_size)) for img in images]
    else:
        images_resized = [resize_image(img, tile_size) for img in images]
    average_colors = [calculate_average_color(img) for img in images_resized]

    print("Creating mosaic...")
    mosaic = Image.new('RGB', target.size)

    for i in range(0, target_width, tile_width):
        for j in range(0, target_height, tile_height):
            target_tile = target.crop((i, j, i + tile_width, j + tile_height))
            target_color = calculate_average_color(target_tile)
            distances = [np.linalg.norm(target_color - color) for color in average_colors]
            sorted_indices = sorted(range(len(distances)), key=lambda k: (distances[k], image_usage[image_files[k]]))
            closest_img_index = sorted_indices[0]
            closest_img = images_resized[closest_img_index]
            image_usage[image_files[closest_img_index]] += 1
            mosaic.paste(closest_img, (i, j))

        print(f"Row {j // tile_height + 1} of {target_height // tile_height} complete.")

    base_output_path = '/content/drive/My Drive/mosaic'  # Replace with your desired path
    output_path = generate_unique_filename(base_output_path)

    # Resize the mosaic to the desired size in inches and set DPI
    mosaic_in_pixels = mosaic.resize((mosaic_size_inches[0]*dpi, mosaic_size_inches[1]*dpi))
    mosaic_in_pixels.save(output_path, 'PNG', dpi=(dpi, dpi))
    print(f"Mosaic created successfully. Saved to {output_path}")

    plt.imshow(mosaic_in_pixels)
    plt.axis('off')
    plt.gcf().set_size_inches(*mosaic_size_inches)
    plt.show()

    return image_usage


In [ ]:
used_tile_filenames = create_mosaic(target_image_path, image_library_folder, num_tiles_x=75, num_tiles_y=50, mosaic_size_inches=(30, 20), dpi=150, output_width=None, output_height=None, apply_sharpening=False)
print("Used tile filenames:", used_tile_filenames)